Importing libraries

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

Reading files into dataframe

In [9]:
df = pd.read_csv("DEVICE_DIRECTORY.csv")
subdf = pd.read_csv("SUB_DEVICE_DIRECTORY.csv")
pd.set_option("display.max_columns", None)

In [10]:
df['RAM_SIZE'].unique()

array([ 2048,   256,  1024,  4096,  6144,  3072,   576,     0,   512,
          70,   768,  1536,    64,   128,  1000,    96,   192,   384,
         288,  6156,   478,  1048,  8192,    16,   280,    32,  2024,
        4092,  4093,  2096,    71,   650,  6015,  2018,  1532,  4000,
       12288,  3048,  8196, 10240,   125,  8190,  3096,  6196, 16384,
         248,   409, 12228, 12888,  6072,  6000, 18432,  6114,  2000,
        3075,   448,    24], dtype=int64)

In [11]:
df.to_csv('DEVICE_DIRECTORY.csv', index=False)


In [12]:
subdf

,ID,DEVICE_ID,BRAND,MODEL,NETTYPE,OS,OS_TYPE,START_DATE,END_DATE,DAYS_USED,AVG_DAYS_USED
0,4397787,35230713,Samsung,Galaxy A12,4G,Android,smartphone,2024-01-01,2024-07-12,193,616.0
1,4129638,35306809,Apple,iPhone SE,4G,iOS,smartphone,2022-04-01,2023-05-31,425,642.5
2,1154183,86887805,OnePlus,Nord N100,4G,Android,smartphone,2021-11-01,2024-07-12,984,564.8
3,670331,86296505,Xiaomi,Redmi 9T,4G,Android,smartphone,2021-05-01,2024-02-29,1034,624.8
4,3074468,35639208,Samsung,Galaxy J7 2017 Dual,4G,Android,smartphone,2021-10-01,2021-10-31,30,653.4
...,...,...,...,...,...,...,...,...,...,...,...
12354469,327793,86324003,Lenovo,A1000 Dual,3G,Android,smartphone,2019-07-01,2019-07-31,30,521.0
12354470,2738649,35480111,Samsung,Galaxy A01 Dual Global,4G,Android,smartphone,2020-09-01,2021-11-30,455,607.9
12354471,1052312,86597302,Alcatel,One Touch Pop C7 Dual,3G,Android,smartphone,2015-08-01,2015-09-30,60,554.9
12354472,4195159,35734105,LG,Optimus L3,3G,Android,smartphone,2017-03-01,2019-02-28,729,468.3


Converting 'DATE' columns into datetime datatype

In [13]:
subdf['START_DATE'] = pd.to_datetime(subdf['START_DATE'])
subdf['END_DATE'] = pd.to_datetime(subdf['END_DATE'])
# subdf.to_csv('SUB_DEVICE_DIRECTORY.csv', index=False)
(subdf[subdf["ID"] == 1052312]).sort_values(by='START_DATE')


,ID,DEVICE_ID,BRAND,MODEL,NETTYPE,OS,OS_TYPE,START_DATE,END_DATE,DAYS_USED,AVG_DAYS_USED
12354471,1052312,86597302,Alcatel,One Touch Pop C7 Dual,3G,Android,smartphone,2015-08-01,2015-09-30,60,554.9
1838851,1052312,35477307,HTC,Desire 530,4G,Android,smartphone,2018-04-01,2018-07-31,121,526.6
10167412,1052312,35847509,Samsung,Galaxy J2 2018 Dual,4G,Android,smartphone,2018-08-01,2019-10-31,456,681.3
2899958,1052312,35858310,Samsung,Galaxy A50 Dual,4G,Android,smartphone,2019-11-01,2023-05-31,1307,773.0
2629479,1052312,86344705,Huawei,P30 lite,4G,Android,smartphone,2023-06-01,2023-11-30,182,663.7
3924239,1052312,35895309,Samsung,Galaxy A6 2018 Dual,4G,Android,smartphone,2023-12-01,2024-07-12,224,808.5


Filter brands from the average days used per brand dataset, sorted in descending order.


In [14]:
avg_days_used_per_brand = subdf.groupby('BRAND')['DAYS_USED'].mean().reset_index()
avg_days_used_per_brand.columns = ['BRAND', 'AVG_DAYS_USED']
pd.set_option("display.max_rows", None)
avg_days_used_per_brand.sort_values(by="AVG_DAYS_USED", ascending=False)
filtered_brands = avg_days_used_per_brand[avg_days_used_per_brand['BRAND'].isin(
    ['Apple', 'Samsung','Nokia','Xiaomi','Huawei','Motorola','Microsoft'])]

In [15]:
subdf.columns
    

Index(['ID', 'DEVICE_ID', 'BRAND', 'MODEL', 'NETTYPE', 'OS', 'OS_TYPE',
       'START_DATE', 'END_DATE', 'DAYS_USED', 'AVG_DAYS_USED'],
      dtype='object')

Addding a new column with the rounded average days used for each model and brand combination in the dataset.


In [17]:
subdf['AVG_DAYS_USED'] = (subdf.groupby(['MODEL', 'BRAND'])['DAYS_USED'].transform('mean')).round(1)
subdf.to_csv('SUB_DEVICE_DIRECTORY.csv', index=False)

Merge `subdf` with `df` on `DEVICE_ID` and `TAC`, including all rows from `subdf` and matching rows from `df`.


In [18]:
merged_df = pd.merge(subdf, df, left_on='DEVICE_ID', right_on='TAC', how='left')
merged_df.sample(6)

,ID,DEVICE_ID,BRAND,MODEL,NETTYPE,OS,OS_TYPE_x,START_DATE,END_DATE,DAYS_USED,AVG_DAYS_USED,TAC,MODEL_NAME,VENDOR_NAME,SUPPORTS_LTE,SUPPORTS_VOLTE,BATTERY_CAPACITY,BATTERY_TYPE,BODY_DEPTH,BODY_HEIGHT,BODY_WIDTH,BODY_TYPE,OS_TYPE_y,OS_VENDOR,CAMERA_FLASHLIGHT,SUPPORTS_MULTISIM,SUPPORTS_ESIM,SUPPORT_5G,SIM_COUNT,CPU_CORES,MULTISIM_MODE,PIXEL_DENSITY,RAM_SIZE,RELEASE_DATE,BAND_1800,BAND_1900,BAND_850,BAND_900
10680424,3060181,35643408,Samsung,Galaxy J5 2017 Dual,4G,Android,smartphone,2017-11-01,2021-04-30,1276,668.6,35643408,Galaxy J5 2017 Dual,Samsung,1,1,3000.0,Li-ion,7.90,146.2,71.3,touch,Android,Google,1,1,0,0,2,8,Standby,282.0,2048,2017-04-01,1,1,1,1
9266001,1344761,35642502,Nokia,N73,3G,Symbian,smartphone,2014-09-01,2016-03-31,577,287.7,35642502,N73,Nokia,0,0,1100.0,Li-polymer,19.00,110.0,49.0,classic,Symbian,Nokia,1,0,0,0,1,1,Standby,167.0,64,2006-07-01,1,1,1,1
11612987,1161228,35921907,Apple,iPhone 7 Plus,4G,iOS,smartphone,2022-04-01,2024-07-12,833,681.9,35921907,iPhone 7 Plus,Apple,1,1,2900.0,Li-ion,7.30,158.2,77.9,touch,iOS,Apple,1,0,0,0,1,8,Standby,401.0,3072,2016-09-01,1,1,1,1
1145138,2469492,35286567,Apple,iPhone 13 Pro Max,5G,iOS,smartphone,2022-09-01,2024-07-12,680,461.6,35286567,iPhone 13 Pro Max,Apple,1,1,4352.0,Li-ion,7.65,160.8,78.1,touch,iOS,Apple,0,1,1,1,1,6,Standby,457.0,8192,2021-11-16,1,1,1,1
8969137,249246,35993004,Samsung,Galaxy SL,3G,Android,smartphone,2014-04-01,2016-04-30,760,369.9,35993004,Galaxy SL,Samsung,0,0,1650.0,Li-ion,10.59,123.7,64.2,touch,Android,Google,0,0,0,0,1,1,Standby,233.0,478,2011-01-01,1,1,1,1
7620579,2546185,35883308,Fly,Stratus 7,3G,Android,smartphone,2019-08-01,2024-07-12,1807,367.3,35883308,Stratus 7,Fly,0,0,1750.0,Li-ion,10.30,135.0,66.0,touch,Android,Google,1,1,0,0,2,8,Standby,218.0,512,2017-07-01,1,0,0,1


In [ ]:
merged_df.head()

,ID,DEVICE_ID,BRAND,MODEL,NETTYPE,OS,OS_TYPE,START_DATE,END_DATE,DAYS_USED,AVG_DAYS_USED,SUPPORTS_LTE,SUPPORTS_VOLTE,BATTERY_CAPACITY,BATTERY_TYPE,BODY_DEPTH,BODY_HEIGHT,BODY_WIDTH,BODY_TYPE,OS_VENDOR,CAMERA_FLASHLIGHT,SUPPORTS_MULTISIM,SUPPORTS_ESIM,SUPPORT_5G,SIM_COUNT,CPU_CORES,MULTISIM_MODE,PIXEL_DENSITY,RAM_SIZE,RELEASE_DATE,BAND_1800,BAND_1900,BAND_850,BAND_900
0,4397787,35230713,Samsung,Galaxy A12,4G,Android,smartphone,2024-01-01,2024-07-12,193,616.0,1,1,5000.0,Li-ion,8.9,164.0,75.80,touch,Google,1,1,0,0,2,8,dual standby,264.0,6156,2020-08-19,1,1,1,1
1,4129638,35306809,Apple,iPhone SE,4G,iOS,smartphone,2022-04-01,2023-05-31,425,642.5,1,0,1642.0,Li-ion,7.6,123.8,58.60,touch,Apple,1,0,0,0,1,2,Standby,326.0,2048,2016-03-01,1,1,1,1
2,1154183,86887805,OnePlus,Nord N100,4G,Android,smartphone,2021-11-01,2024-07-12,984,564.8,1,1,5000.0,Li-polymer,8.5,164.9,75.10,touch,Google,1,1,0,0,2,8,dual standby,269.0,6156,2020-05-21,1,1,1,1
3,670331,86296505,Xiaomi,Redmi 9T,4G,Android,smartphone,2021-05-01,2024-02-29,1034,624.8,1,1,6000.0,Li-polymer,9.6,162.3,77.28,touch,Google,0,1,0,0,2,8,Standby,395.0,4096,2021-01-01,1,1,1,1
4,3074468,35639208,Samsung,Galaxy J7 2017 Dual,4G,Android,smartphone,2021-10-01,2021-10-31,30,653.4,1,1,3600.0,Li-ion,7.8,152.4,74.70,touch,Google,1,1,0,0,2,8,Standby,401.0,3072,2017-07-01,1,1,1,1


In [22]:
merged_df.sample(12)

,ID,DEVICE_ID,BRAND,MODEL,NETTYPE,OS,OS_TYPE_x,START_DATE,END_DATE,DAYS_USED,AVG_DAYS_USED,TAC,MODEL_NAME,VENDOR_NAME,SUPPORTS_LTE,SUPPORTS_VOLTE,BATTERY_CAPACITY,BATTERY_TYPE,BODY_DEPTH,BODY_HEIGHT,BODY_WIDTH,BODY_TYPE,OS_TYPE_y,OS_VENDOR,CAMERA_FLASHLIGHT,SUPPORTS_MULTISIM,SUPPORTS_ESIM,SUPPORT_5G,SIM_COUNT,CPU_CORES,MULTISIM_MODE,PIXEL_DENSITY,RAM_SIZE,RELEASE_DATE,BAND_1800,BAND_1900,BAND_850,BAND_900
6891253,1343722,35440906,Apple,iPhone 6,4G,iOS,smartphone,2017-10-01,2019-01-31,487,703.6,35440906,iPhone 6,Apple,1,1,1810.00,Li-ion,6.9,138.10,67.00,touch,iOS,Apple,1,0,0,0,1,2,Standby,326.00,1024,2014-07-10,1,1,1,1
11781184,1640018,35888208,Vertex,Impress Luck,3G,Android,smartphone,2018-02-01,2018-07-31,180,385.4,35888208,Impress Luck,Vertex,0,0,2200.00,Li-ion,9.0,141.00,71.00,touch,Android,Google,1,1,0,0,2,8,dual standby,196.00,1024,2017-04-04,1,1,1,1
9054003,3777638,86359503,Shenzhen Fortuneship Technology Co Ltd,SS335,3G,Android,smartphone,2019-12-01,2020-10-31,335,390.2,86359503,SS335,Shenzhen Fortuneship Technology Co Ltd,0,0,3257.14,Li-ion,9.0,147.80,71.88,touch,Android,Google,0,1,0,0,1,8,Standby,338.94,4096,2016-10-20,1,1,1,1
969938,364126,86982700,Alcatel,918D Dual,3G,Android,smartphone,2014-04-01,2016-02-29,699,396.1,86982700,918D Dual,Alcatel,0,0,1300.00,Li-ion,12.1,112.00,58.60,touch,Android,Google,0,1,0,0,2,1,Standby,180.00,256,2011-08-01,1,1,1,1
5516455,1831060,35172305,Samsung,Wave 525,2G,Bada,smartphone,2014-04-01,2021-03-31,2556,504.4,35172305,Wave 525,Samsung,0,0,1200.00,Li-ion,11.7,109.50,55.00,touch,Bada,Samsung,0,0,0,0,1,1,Standby,146.00,4096,2010-04-01,1,1,1,1
12058012,176616,35962304,Samsung,Galaxy Omnia W,3G,Windows,smartphone,2013-09-01,2015-01-31,517,413.6,35962304,Galaxy Omnia W,Samsung,0,0,1500.00,Li-ion,10.9,115.97,58.80,touch,Windows Phone,Microsoft,1,0,0,0,1,8,Standby,252.00,512,2011-10-01,1,1,1,1
10004541,1760015,35932204,Nokia,C7-00,3G,Symbian,smartphone,2014-05-01,2014-11-30,213,393.4,35932204,C7-00,Nokia,0,0,1200.00,Li-ion,10.5,117.30,56.80,touch,Symbian,Nokia,1,0,0,0,1,1,Standby,210.00,256,2010-10-01,1,1,1,1
5358632,3361493,35435908,Samsung,Galaxy S8 Plus Dual,4G,Android,smartphone,2019-12-01,2022-01-31,792,762.0,35435908,Galaxy S8 Plus Dual,Samsung,1,1,3500.00,Li-ion,8.1,159.50,73.40,touch,Android,Google,1,1,0,0,2,8,Standby,531.00,4096,2017-04-01,1,1,1,1
2595193,4283206,35577408,Samsung,Galaxy J5 2016 LTE Dual,4G,Android,smartphone,2021-12-01,2024-07-12,954,746.2,35577408,Galaxy J5 2016 LTE Dual,Samsung,1,0,4300.00,Li-ion,7.9,145.70,72.30,touch,Android,Google,1,1,0,0,2,4,Standby,282.00,2048,2016-04-01,1,1,1,1
7101706,973028,35881510,Samsung,Galaxy S10 Plus Dual,4G,Android,smartphone,2021-07-01,2024-03-31,1004,689.4,35881510,Galaxy S10 Plus Dual,Samsung,1,1,4100.00,Li-ion,7.8,157.60,74.10,touch,Android,Google,1,1,0,0,2,8,dual standby,526.00,8192,2019-01-01,1,1,1,1


In [28]:
merged_df.columns

Index(['ID', 'DEVICE_ID', 'BRAND', 'MODEL', 'NETTYPE', 'OS', 'OS_TYPE_x',
       'START_DATE', 'END_DATE', 'DAYS_USED', 'AVG_DAYS_USED', 'TAC',
       'MODEL_NAME', 'VENDOR_NAME', 'SUPPORTS_LTE', 'SUPPORTS_VOLTE',
       'BATTERY_CAPACITY', 'BATTERY_TYPE', 'BODY_DEPTH', 'BODY_HEIGHT',
       'BODY_WIDTH', 'BODY_TYPE', 'OS_TYPE_y', 'OS_VENDOR',
       'CAMERA_FLASHLIGHT', 'SUPPORTS_MULTISIM', 'SUPPORTS_ESIM', 'SUPPORT_5G',
       'SIM_COUNT', 'CPU_CORES', 'MULTISIM_MODE', 'PIXEL_DENSITY', 'RAM_SIZE',
       'RELEASE_DATE', 'BAND_1800', 'BAND_1900', 'BAND_850', 'BAND_900',
       'POST_RELEASE_PURCHASE'],
      dtype='object')

Calculating the post-release purchase period by subtracting the `RELEASE_DATE` from the `START_DATE` and storing it in a new column.


In [26]:
merged_df['POST_RELEASE_PURCHASE'] = pd.to_datetime(merged_df['START_DATE']) - pd.to_datetime(merged_df['RELEASE_DATE'])

In [27]:
merged_df.head()

,ID,DEVICE_ID,BRAND,MODEL,NETTYPE,OS,OS_TYPE_x,START_DATE,END_DATE,DAYS_USED,AVG_DAYS_USED,TAC,MODEL_NAME,VENDOR_NAME,SUPPORTS_LTE,SUPPORTS_VOLTE,BATTERY_CAPACITY,BATTERY_TYPE,BODY_DEPTH,BODY_HEIGHT,BODY_WIDTH,BODY_TYPE,OS_TYPE_y,OS_VENDOR,CAMERA_FLASHLIGHT,SUPPORTS_MULTISIM,SUPPORTS_ESIM,SUPPORT_5G,SIM_COUNT,CPU_CORES,MULTISIM_MODE,PIXEL_DENSITY,RAM_SIZE,RELEASE_DATE,BAND_1800,BAND_1900,BAND_850,BAND_900,POST_RELEASE_PURCHASE
0,4397787,35230713,Samsung,Galaxy A12,4G,Android,smartphone,2024-01-01,2024-07-12,193,616.0,35230713,Galaxy A12,Samsung,1,1,5000.0,Li-ion,8.9,164.0,75.80,touch,Android,Google,1,1,0,0,2,8,dual standby,264.0,6156,2020-08-19,1,1,1,1,1230 days
1,4129638,35306809,Apple,iPhone SE,4G,iOS,smartphone,2022-04-01,2023-05-31,425,642.5,35306809,iPhone SE,Apple,1,0,1642.0,Li-ion,7.6,123.8,58.60,touch,iOS,Apple,1,0,0,0,1,2,Standby,326.0,2048,2016-03-01,1,1,1,1,2222 days
2,1154183,86887805,OnePlus,Nord N100,4G,Android,smartphone,2021-11-01,2024-07-12,984,564.8,86887805,Nord N100,OnePlus,1,1,5000.0,Li-polymer,8.5,164.9,75.10,touch,Android,Google,1,1,0,0,2,8,dual standby,269.0,6156,2020-05-21,1,1,1,1,529 days
3,670331,86296505,Xiaomi,Redmi 9T,4G,Android,smartphone,2021-05-01,2024-02-29,1034,624.8,86296505,Redmi 9T,Xiaomi,1,1,6000.0,Li-polymer,9.6,162.3,77.28,touch,Android,Google,0,1,0,0,2,8,Standby,395.0,4096,2021-01-01,1,1,1,1,120 days
4,3074468,35639208,Samsung,Galaxy J7 2017 Dual,4G,Android,smartphone,2021-10-01,2021-10-31,30,653.4,35639208,Galaxy J7 2017 Dual,Samsung,1,1,3600.0,Li-ion,7.8,152.4,74.70,touch,Android,Google,1,1,0,0,2,8,Standby,401.0,3072,2017-07-01,1,1,1,1,1553 days


Converting the `POST_RELEASE_PURCHASE` column to integers by extracting the number of days as a string and then converting it to an integer.


In [35]:
merged_df['POST_RELEASE_PURCHASE'] = merged_df['POST_RELEASE_PURCHASE'].astype(str).str.extract('(\d+)').astype(int)


In [36]:
merged_df = merged_df.rename(columns={'OS_TYPE_x': 'OS_TYPE'})


In [37]:
merged_df = merged_df[['ID', 'DEVICE_ID', 'BRAND', 'MODEL', 'NETTYPE', 'OS', 'OS_TYPE', 'OS_VENDOR',
        'RELEASE_DATE','START_DATE', 'END_DATE', 'POST_RELEASE_PURCHASE', 'DAYS_USED', 'AVG_DAYS_USED', 'SUPPORTS_LTE',
        'SUPPORTS_VOLTE', 'BAND_1800', 'BAND_1900', 'BAND_850', 'BAND_900',
        'CAMERA_FLASHLIGHT', 'SUPPORTS_MULTISIM', 'SUPPORTS_ESIM', 'SUPPORT_5G',
        'RAM_SIZE', 'SIM_COUNT', 'CPU_CORES', 'BATTERY_CAPACITY', 'BATTERY_TYPE', 
        'BODY_DEPTH', 'BODY_HEIGHT', 'BODY_WIDTH', 'BODY_TYPE',
         'MULTISIM_MODE', 'PIXEL_DENSITY']]

In [41]:
merged_df.shape

(12354474, 35)

Converting the `merged_df` DataFrame to a Dask DataFrame with 12 partitions and saving it as a single CSV file named `MERGED_DATA_PART.csv`.


In [40]:
ddf = dd.from_pandas(merged_df, npartitions=12) 
ddf.to_csv('MERGED_DATA_PART.csv', index=False, single_file=True)

Data saved successfully using Dask.


In [42]:
ddf.head()

,ID,DEVICE_ID,BRAND,MODEL,NETTYPE,OS,OS_TYPE,OS_VENDOR,RELEASE_DATE,START_DATE,END_DATE,POST_RELEASE_PURCHASE,DAYS_USED,AVG_DAYS_USED,SUPPORTS_LTE,SUPPORTS_VOLTE,BAND_1800,BAND_1900,BAND_850,BAND_900,CAMERA_FLASHLIGHT,SUPPORTS_MULTISIM,SUPPORTS_ESIM,SUPPORT_5G,RAM_SIZE,SIM_COUNT,CPU_CORES,BATTERY_CAPACITY,BATTERY_TYPE,BODY_DEPTH,BODY_HEIGHT,BODY_WIDTH,BODY_TYPE,MULTISIM_MODE,PIXEL_DENSITY
0,4397787,35230713,Samsung,Galaxy A12,4G,Android,smartphone,Google,2020-08-19,2024-01-01,2024-07-12,1230,193,616.0,1,1,1,1,1,1,1,1,0,0,6156,2,8,5000.0,Li-ion,8.9,164.0,75.80,touch,dual standby,264.0
1,4129638,35306809,Apple,iPhone SE,4G,iOS,smartphone,Apple,2016-03-01,2022-04-01,2023-05-31,2222,425,642.5,1,0,1,1,1,1,1,0,0,0,2048,1,2,1642.0,Li-ion,7.6,123.8,58.60,touch,Standby,326.0
2,1154183,86887805,OnePlus,Nord N100,4G,Android,smartphone,Google,2020-05-21,2021-11-01,2024-07-12,529,984,564.8,1,1,1,1,1,1,1,1,0,0,6156,2,8,5000.0,Li-polymer,8.5,164.9,75.10,touch,dual standby,269.0
3,670331,86296505,Xiaomi,Redmi 9T,4G,Android,smartphone,Google,2021-01-01,2021-05-01,2024-02-29,120,1034,624.8,1,1,1,1,1,1,0,1,0,0,4096,2,8,6000.0,Li-polymer,9.6,162.3,77.28,touch,Standby,395.0
4,3074468,35639208,Samsung,Galaxy J7 2017 Dual,4G,Android,smartphone,Google,2017-07-01,2021-10-01,2021-10-31,1553,30,653.4,1,1,1,1,1,1,1,1,0,0,3072,2,8,3600.0,Li-ion,7.8,152.4,74.70,touch,Standby,401.0


In [43]:
total_data = pd.read_csv('MERGED_DATA_PART.csv')

In [44]:
total_data.head()

,ID,DEVICE_ID,BRAND,MODEL,NETTYPE,OS,OS_TYPE,OS_VENDOR,RELEASE_DATE,START_DATE,END_DATE,POST_RELEASE_PURCHASE,DAYS_USED,AVG_DAYS_USED,SUPPORTS_LTE,SUPPORTS_VOLTE,BAND_1800,BAND_1900,BAND_850,BAND_900,CAMERA_FLASHLIGHT,SUPPORTS_MULTISIM,SUPPORTS_ESIM,SUPPORT_5G,RAM_SIZE,SIM_COUNT,CPU_CORES,BATTERY_CAPACITY,BATTERY_TYPE,BODY_DEPTH,BODY_HEIGHT,BODY_WIDTH,BODY_TYPE,MULTISIM_MODE,PIXEL_DENSITY
0,4397787,35230713,Samsung,Galaxy A12,4G,Android,smartphone,Google,2020-08-19,2024-01-01,2024-07-12,1230,193,616.0,1,1,1,1,1,1,1,1,0,0,6156,2,8,5000.0,Li-ion,8.9,164.0,75.80,touch,dual standby,264.0
1,4129638,35306809,Apple,iPhone SE,4G,iOS,smartphone,Apple,2016-03-01,2022-04-01,2023-05-31,2222,425,642.5,1,0,1,1,1,1,1,0,0,0,2048,1,2,1642.0,Li-ion,7.6,123.8,58.60,touch,Standby,326.0
2,1154183,86887805,OnePlus,Nord N100,4G,Android,smartphone,Google,2020-05-21,2021-11-01,2024-07-12,529,984,564.8,1,1,1,1,1,1,1,1,0,0,6156,2,8,5000.0,Li-polymer,8.5,164.9,75.10,touch,dual standby,269.0
3,670331,86296505,Xiaomi,Redmi 9T,4G,Android,smartphone,Google,2021-01-01,2021-05-01,2024-02-29,120,1034,624.8,1,1,1,1,1,1,0,1,0,0,4096,2,8,6000.0,Li-polymer,9.6,162.3,77.28,touch,Standby,395.0
4,3074468,35639208,Samsung,Galaxy J7 2017 Dual,4G,Android,smartphone,Google,2017-07-01,2021-10-01,2021-10-31,1553,30,653.4,1,1,1,1,1,1,1,1,0,0,3072,2,8,3600.0,Li-ion,7.8,152.4,74.70,touch,Standby,401.0
